In [56]:
import pandas as pd

In [57]:
alibaba_batch_task_df = pd.read_csv('data/alibaba/batch_task_chunk_10000.csv')
alibaba_batch_task_df = alibaba_batch_task_df[alibaba_batch_task_df['status'] == 'Terminated']
alibaba_batch_task_df = alibaba_batch_task_df.sort_values(by='start_time', ascending=True)
alibaba_batch_task_df.reset_index(drop=True, inplace=True)
alibaba_batch_task_df = alibaba_batch_task_df.drop(columns=['status','job_name'])
alibaba_batch_task_df.head()

,task_name,instance_num,task_type,start_time,end_time,plan_cpu,plan_mem
0,M2,40.0,1,86207,86485,50.0,0.39
1,J7_1_2_6,1.0,1,86207,86622,50.0,0.20
2,M3,1.0,1,86207,86210,50.0,0.20
3,R4_3,1.0,1,86207,86212,50.0,0.20
4,M5_4,17.0,1,86207,86300,50.0,0.39


In [58]:
# Use pd.get_dummies for one-hot encoding
one_hot_encoded = pd.get_dummies(alibaba_batch_task_df['task_type'], prefix='task_type')
one_hot_encoded = one_hot_encoded.astype(int)
alibaba_batch_task_df = pd.concat([alibaba_batch_task_df, one_hot_encoded], axis=1)

alibaba_batch_task_df.head()

,task_name,instance_num,task_type,start_time,end_time,plan_cpu,plan_mem,task_type_1,task_type_3,task_type_4,task_type_5,task_type_6,task_type_8,task_type_9,task_type_10,task_type_11,task_type_12
0,M2,40.0,1,86207,86485,50.0,0.39,1,0,0,0,0,0,0,0,0,0
1,J7_1_2_6,1.0,1,86207,86622,50.0,0.20,1,0,0,0,0,0,0,0,0,0
2,M3,1.0,1,86207,86210,50.0,0.20,1,0,0,0,0,0,0,0,0,0
3,R4_3,1.0,1,86207,86212,50.0,0.20,1,0,0,0,0,0,0,0,0,0
4,M5_4,17.0,1,86207,86300,50.0,0.39,1,0,0,0,0,0,0,0,0,0


In [59]:
import pandas as pd

def determine_task_category(task_name):
    if task_name == 'MergeTask':
        return '1' #merge
    elif task_name.startswith('M'):
        return '2'
    elif task_name.startswith('J'):
        return '3'
    elif task_name.startswith('R'):
        return '4'
    else:
        return '0' #independent

alibaba_batch_task_df['task_category'] = alibaba_batch_task_df['task_name'].apply(determine_task_category)
one_hot_encoded = pd.get_dummies(alibaba_batch_task_df['task_category'], prefix='task_category')
one_hot_encoded = one_hot_encoded.astype(int)
alibaba_batch_task_df = pd.concat([alibaba_batch_task_df, one_hot_encoded], axis=1)

alibaba_batch_task_df.head()

,task_name,instance_num,task_type,start_time,end_time,plan_cpu,plan_mem,task_type_1,task_type_3,task_type_4,...,task_type_9,task_type_10,task_type_11,task_type_12,task_category,task_category_0,task_category_1,task_category_2,task_category_3,task_category_4
0,M2,40.0,1,86207,86485,50.0,0.39,1,0,0,...,0,0,0,0,2,0,0,1,0,0
1,J7_1_2_6,1.0,1,86207,86622,50.0,0.20,1,0,0,...,0,0,0,0,3,0,0,0,1,0
2,M3,1.0,1,86207,86210,50.0,0.20,1,0,0,...,0,0,0,0,2,0,0,1,0,0
3,R4_3,1.0,1,86207,86212,50.0,0.20,1,0,0,...,0,0,0,0,4,0,0,0,0,1
4,M5_4,17.0,1,86207,86300,50.0,0.39,1,0,0,...,0,0,0,0,2,0,0,1,0,0


In [60]:
# Split task_name by "_" and extract the numbers
alibaba_batch_task_df['task_name'] = alibaba_batch_task_df['task_name'].str.replace('[RJM]', '', regex=True)
alibaba_batch_task_df['numbers'] = alibaba_batch_task_df['task_name'].apply(lambda x: [int(num) for num in x.split('_') if num.isdigit()])
alibaba_batch_task_df['main_task'] = alibaba_batch_task_df['numbers'].apply(lambda x: x[0] if len(x) > 0 else 0)

main_task_col = alibaba_batch_task_df.pop('main_task')
alibaba_batch_task_df.insert(7, 'main_task', main_task_col)

# Create columns for job_1, job_2, job_3...
for i in range(1, 58):
    col_name = f'job_{i}'
    alibaba_batch_task_df[col_name] = alibaba_batch_task_df['numbers'].apply(lambda x: 1 if i in x else 0)

alibaba_batch_task_df.drop(columns=['numbers'], inplace=True)
alibaba_batch_task_df.drop(columns=['task_name'], inplace=True)
alibaba_batch_task_df.drop(columns=['task_type'], inplace=True)
alibaba_batch_task_df.drop(columns=['task_category'], inplace=True)

alibaba_batch_task_df.head()

,instance_num,start_time,end_time,plan_cpu,plan_mem,main_task,task_type_1,task_type_3,task_type_4,task_type_5,...,job_48,job_49,job_50,job_51,job_52,job_53,job_54,job_55,job_56,job_57
0,40.0,86207,86485,50.0,0.39,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,86207,86622,50.0,0.20,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,86207,86210,50.0,0.20,3,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,86207,86212,50.0,0.20,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17.0,86207,86300,50.0,0.39,5,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
# One-hot encode main_task
main_task_encoded = pd.get_dummies(alibaba_batch_task_df['main_task'], prefix='main_task').astype(int)

# One-hot encode plan_cpu
plan_cpu_encoded = pd.get_dummies(alibaba_batch_task_df['plan_cpu'], prefix='plan_cpu').astype(int)

# One-hot encode plan_mem
plan_mem_encoded = pd.get_dummies(alibaba_batch_task_df['plan_mem'], prefix='plan_mem').astype(int)

# Concatenate the one-hot encoded columns to the original DataFrame
alibaba_batch_task_df = pd.concat([alibaba_batch_task_df, main_task_encoded, plan_cpu_encoded, plan_mem_encoded], axis=1)

# Pop the older columns
alibaba_batch_task_df.pop('main_task')
alibaba_batch_task_df.pop('plan_cpu')
alibaba_batch_task_df.pop('plan_mem')

# Display the resulting DataFrame
alibaba_batch_task_df.head()

,instance_num,start_time,end_time,task_type_1,task_type_3,task_type_4,task_type_5,task_type_6,task_type_8,task_type_9,...,plan_mem_0.77,plan_mem_0.79,plan_mem_0.83,plan_mem_0.96,plan_mem_0.97,plan_mem_1.34,plan_mem_1.47,plan_mem_1.53,plan_mem_1.72,plan_mem_2.35
0,40.0,86207,86485,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,86207,86622,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,86207,86210,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,86207,86212,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17.0,86207,86300,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
alibaba_batch_task_df['instance_num'] = alibaba_batch_task_df['instance_num'].astype(int)
alibaba_batch_task_df.dropna(inplace=True)
alibaba_batch_task_df.to_csv('data/alibaba/batch_task_chunk_preprocessed_10000.csv', index=False)

In [77]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Assuming alibaba_batch_task_df is already loaded
features = alibaba_batch_task_df.drop(columns=['instance_num', 'start_time', 'end_time'])

# Convert data to PyTorch tensors
features_tensor = torch.tensor(features.values, dtype=torch.float32)

# Split data into training and validation sets
train_size = int(0.8 * len(features_tensor))
val_size = len(features_tensor) - train_size
train_dataset, val_dataset = random_split(TensorDataset(features_tensor, features_tensor), [train_size, val_size])
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

input_dim = features.shape[1]
encoding_dim = 8  # Adjust based on your needs
autoencoder = Autoencoder(input_dim, encoding_dim)

# Training the autoencoder
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
num_epochs = 100

for epoch in range(num_epochs):
    autoencoder.train()
    for data in train_dataloader:
        inputs, _ = data
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    autoencoder.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_dataloader:
            inputs, _ = data
            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            val_loss += loss.item()

    val_loss /= len(val_dataloader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {val_loss:.4f}')

# Encode the data
encoded_data = autoencoder.encoder(features_tensor).detach().numpy()

# Decode the data
decoded_data = autoencoder.decoder(torch.tensor(encoded_data, dtype=torch.float32)).detach().numpy()

# Reconstruct the original DataFrame with decoded features
reconstructed_data = pd.DataFrame(decoded_data, columns=features.columns)


Epoch [1/100], Validation Loss: 0.0306
Epoch [2/100], Validation Loss: 0.0254
Epoch [3/100], Validation Loss: 0.0228
Epoch [4/100], Validation Loss: 0.0196
Epoch [5/100], Validation Loss: 0.0168
Epoch [6/100], Validation Loss: 0.0143
Epoch [7/100], Validation Loss: 0.0124
Epoch [8/100], Validation Loss: 0.0110
Epoch [9/100], Validation Loss: 0.0098
Epoch [10/100], Validation Loss: 0.0088
Epoch [11/100], Validation Loss: 0.0081
Epoch [12/100], Validation Loss: 0.0075
Epoch [13/100], Validation Loss: 0.0070
Epoch [14/100], Validation Loss: 0.0065
Epoch [15/100], Validation Loss: 0.0062
Epoch [16/100], Validation Loss: 0.0059
Epoch [17/100], Validation Loss: 0.0056
Epoch [18/100], Validation Loss: 0.0053
Epoch [19/100], Validation Loss: 0.0051
Epoch [20/100], Validation Loss: 0.0050
Epoch [21/100], Validation Loss: 0.0048
Epoch [22/100], Validation Loss: 0.0047
Epoch [23/100], Validation Loss: 0.0046
Epoch [24/100], Validation Loss: 0.0044
Epoch [25/100], Validation Loss: 0.0043
Epoch [26

In [78]:
# Define the group identifiers
group_identifiers = ['task_type', 'task_category', 'main_task', 'plan_mem', 'plan_cpu']

decoded_data_rounded = reconstructed_data.copy()


# Iterate over each group identifier
for group_id in group_identifiers:
    # Get the list of columns that contain the group identifier
    columns = [i-(alibaba_batch_task_df.columns.size-decoded_data.shape[1]) for i, col in enumerate(alibaba_batch_task_df.columns) if group_id in col]
    decoded_data_rounded.iloc[:, columns] = decoded_data_rounded.iloc[:, columns].apply(lambda x: (x == x.max()), axis=1).astype(int)

columns = [i-(alibaba_batch_task_df.columns.size-decoded_data.shape[1]) for i, col in enumerate(alibaba_batch_task_df.columns) if 'job' in col]
reconstructed_data.iloc[:, columns] = reconstructed_data.iloc[:, columns].map(lambda x: 1 if x > 0.5 else 0)


decoded_data_rounded.iloc[:, columns] = reconstructed_data.iloc[:, columns]

final_df = pd.concat([alibaba_batch_task_df[['instance_num', 'start_time', 'end_time']], pd.DataFrame(decoded_data_rounded, columns=features.columns).astype(int)], axis=1)
final_df.to_csv('decoded_data.csv', index=False)